Getting all wikihow article urls from JSON dump

In [1]:
import os, bs4, srsly, requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
cwd = os.getcwd()
dump_path = os.path.join(cwd, 'wikihow_dump_json')
articles = os.listdir(dump_path)

df = pd.DataFrame()
df['title'] = [srsly.read_json(os.path.join(dump_path, f)).get('title', None) for f in articles]
df['url'] = [srsly.read_json(os.path.join(dump_path, f)).get('url', None) for f in articles]
df['video'] = ''
df['category'] = [srsly.read_json(os.path.join(dump_path, f)).get('category_hierarchy', None)[0] if srsly.read_json(os.path.join(dump_path, f)).get('category_hierarchy', None) else None for f in articles]

In [8]:
df['accessed'] = ''
df.to_csv('wikihow_videos.csv', index=False)

Using beautiful soup to get the youtube link to all of the videos for articles we don't have videos for yet

In [51]:
df = pd.read_csv('wikihow_videos.csv')

In [53]:
cwd = os.getcwd()
wikihow_url_videos = []
for v in df.video.tolist():
    if str(v).__contains__('http'):
        wikihow_url_videos.append(v)

num_videos = len(wikihow_url_videos)
print(f'Num videos before = {num_videos}')
count = len(df) - num_videos
isna = df.video.isna()
for i in df.index:
    if isna[i]:
        URL = df['url'][i]
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        embed_video = soup.find(id='youtube_embed_iframe')
        if embed_video:
            embedded_url = embed_video.attrs.get('data-src', 'NO-DATA-SRC-ATTR')
            df['video'][i] = embedded_url
            num_videos+=1
        else:
            df['video'][i] = 'NO-EMBED-VIDEO'
    count-=1
    print(count)
    if count % 100 == 0:
        df.to_csv(os.path.join(cwd, 'wikihow_articles_with_video_links.csv'), index = False)

print(f'Num videos after = {num_videos}')

Num videos before = 2100
110398
110397
110396
110395
110394
110393
110392
110391
110390
110389
110388
110387
110386
110385
110384
110383
110382
110381
110380
110379
110378
110377
110376
110375
110374
110373
110372
110371
110370
110369
110368
110367
110366
110365
110364
110363
110362
110361
110360
110359
110358
110357
110356
110355
110354
110353
110352
110351
110350
110349
110348
110347
110346
110345
110344
110343
110342
110341
110340
110339
110338
110337
110336
110335
110334
110333
110332
110331
110330
110329
110328
110327
110326
110325
110324
110323
110322
110321
110320
110319
110318
110317
110316
110315
110314
110313
110312
110311
110310
110309
110308
110307
110306
110305
110304
110303
110302
110301
110300
110299
110298
110297
110296
110295
110294
110293
110292
110291
110290
110289
110288
110287
110286
110285
110284
110283
110282
110281
110280
110279
110278
110277
110276
110275
110274
110273
110272
110271
110270
110269
110268
110267
110266
110265
110264
110263
110262
110261
110260
11

ConnectionError: HTTPSConnectionPool(host='www.wikihow.com', port=443): Max retries exceeded with url: /Introduce-Cereal-to-a-Baby (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff706a70b20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [19]:
cats = {'no-category':0}
for i in df.index:
    if df['category'][i] != '':
        if 'http' in str(df['video'][i]):
            if df['category'][i] not in cats:
                cats[df['category'][i]] = 0
            cats[df['category'][i]] +=1
    else: cats['no-category']+=1

print(cats)

{'no-category': 0, 'Education and Communications': 108, 'Family Life': 18, 'Youth': 32, 'Pets and Animals': 60, 'Sports and Fitness': 59, 'Personal Care and Style': 127, 'Arts and Entertainment': 103, 'Health': 159, 'Hobbies and Crafts': 186, 'Finance and Business': 68, 'Home and Garden': 212, 'Relationships': 47, 'Computers and Electronics': 188, 'Food and Entertaining': 296, 'Cars & Other Vehicles': 46, 'Philosophy and Religion': 12, 'Holidays and Traditions': 12, 'Work World': 23, 'Travel': 5, nan: 2}
